In [6]:
import ipywidgets as w
import os
import python_modules.commands as cmd
import pandas as pd
import sys
import folium
import numpy as np
import datetime
import time
from IPython.display import Image
import emoji
import tabulate

# Global variables
root = os.getcwd()
stdout = w.Output()

# Default content
config = cmd.read_config(root) 
questions = cmd.read_questions(root)
tuto = cmd.read_tuto_screen(root)
inscr_form_opt = cmd.read_inscr_form(root)

#html layout
err_mess = " style='color:#ed6d6d; background-color:#111111;'"
sc_title = " style='font-size:40px; padding-top:30px; padding-bottom:30px; background-color:#111111'"

# user infos
glob_usr_mail = ""

In [7]:

class auth_screen:
    def __init__(self):
        # Styles
        style = {'description_width': 'initial'}
        
        # Label Widgets
        self.screen_title = w.HTML("<center"+sc_title+">Connexion</center>")
        
        # Interactive Widgets
        self.username_entry = w.Text(value="@gmail.com", description="Adresse mail", style=style, layout=w.Layout(width="260px"))
        self.userpw_entry = w.Password(description="Mot de passe", style=style, layout=w.Layout(width="270px"))
        self.connect_button = w.Button(description=emoji.emojize(':green_circle:')+" Se connecter ")
        self.reset_pw = w.Button(description="Mot de passe oublié")
        self.create_account_but = w.Button(description="Créer un compte")
        
        
        # Actions on buttons
        self.connect_button.on_click(self.callback_connect)
        self.reset_pw.on_click(self.callback_resetPW)
        self.create_account_but.on_click(self.callback_create)
        
        #Images
        self.solo_pote = w.HTML("<center><img src='solo_pote.png' width=150 height=150 align='center'/></center>")
        
    def init_mess(self):
        # messages        
        self.waiting_mess = w.HTML('<center'+err_mess+'>Connection en cours ...</center>')
        self.mail_not_valid = w.HTML("<center"+err_mess+">Veuillez entrer une adresse Gmail valide (...@gmail.com)</center>")
        self.mail_send = w.HTML("<center"+err_mess+">Nous vous avons envoyé un Email de réinitialisation à " + str(self.username_entry.value) + "</center>")
        self.wrong_pw = w.HTML("<center"+err_mess+">Votre mot de passe est incorrect</center>")
        self.unknow_user = w.HTML("<center"+err_mess+">Votre adresse mail n'est pas reconnue</center>")
        self.already_token = w.HTML("<center"+err_mess+">Vous devez d'abord réinitialiser votre mot de passe</center>")
        self.better_create_account = w.HTML("<center"+err_mess+">Votre adresse mail n'est pas reconnue. Vous devriez d'abord créer un compte.</center>")
        
    def show(self):
        self.__init__()
        self.init_mess()
        
        box_list = [self.screen_title, self.username_entry, self.userpw_entry, self.connect_button, [self.create_account_but, self.reset_pw]]
        out = cmd.center_widget(box_list)
        
        with stdout:
            display(self.solo_pote)
            display(out)
        display(stdout)
        
    
    def show_mess(self, mess):
        if mess == 'wait':
            with stdout:
                self.hide_mess()
                display(self.waiting_mess)
        if mess == 'not_gmail':
            with stdout:
                self.hide_mess()
                display(self.mail_not_valid)
        if mess == 'mail_sent':
            with stdout:
                self.hide_mess()
                display(self.mail_send)
        if mess == 'wrong_pw':
            with stdout:
                self.hide_mess()
                display(self.wrong_pw)
        if mess == 'unknow_mail':
            with stdout:
                self.hide_mess()
                display(self.unknow_user)
        if mess == "token":
            with stdout:
                self.hide_mess()
                display(self.already_token)
        if mess == "better_create_account":
            with stdout:
                self.hide_mess()
                display(self.better_create_account)
        
    def hide(self):
        self.hide_mess()
        self.username_entry.close()
        self.userpw_entry.close()
        self.connect_button.close()
        self.reset_pw.close()
        self.create_account_but.close()
        self.screen_title.close()
        self.solo_pote.close()
        
    
    def hide_mess(self):
        self.waiting_mess.close()
        self.mail_not_valid.close()
        self.mail_send.close()
        self.wrong_pw.close()
        self.unknow_user.close()
        self.already_token.close()
        self.better_create_account.close()
        self.init_mess()
        

    def callback_connect(self, b):
        self.user_info = {"ID" : self.username_entry.value, "PW" : self.userpw_entry.value}
        

        #Check that the email adresse is correct 
        if not "@gmail.com" in self.user_info['ID']:
            self.show_mess('not_gmail')
            return

        #Go to DB to authentify user
        bdd = cmd.refresh(root) 
        #print("saved mail", self.user_info['ID'])
        if self.user_info['ID'] in bdd.index.values.tolist():
            if bdd.loc[self.user_info['ID'], 'USER_PW'] == self.user_info['PW']:

                #Redirect to either question or main_screen
                self.show_mess('wait')
                
                if bdd.loc[self.user_info['ID'], 'IS_POLYTECH'] != 'oui':
                    # User should answer the questions and see the tuto before acessing main_screen
                    
                    ##### ACCESS TO SIGNIN_SCREEN #####
                    self.hide()
                    sign_sc.show()
                    ###################################
                    
                else:
                    
                    ##### ACCESS TO MAIN_SCREEN #####
                    glob_usr_mail = self.user_info['ID']
                    self.user_id = self.user_info
                    self.hide()
                    main_sc.show()
                    #################################

            else:
                self.show_mess('wrong_pw')
                return
        else:
            self.show_mess('unknow_mail')
            return
    
    def callback_resetPW(self, b):
        self.user_info = {"ID" : self.username_entry.value, "PW" : self.userpw_entry.value}
        
        # Display informing user to check out his mailbox
        receiver = self.user_info["ID"]
        if not cmd.check_mail_domain(receiver, config):
            self.show_mess('not_gmail')
            return
        
        bdd = cmd.refresh(root)
        # If user is not in BDD propose to create_account
        if not receiver in bdd.index:
            self.show_mess("better_create_account")
            return
        
        #Check that the password has changed before authorizing another password reset 
        print('token <' + str(bdd.loc[receiver, "TOKEN"]) + '>')
        try:
            a = str(bdd.loc[receiver, "TOKEN"])
            if len(a) > 3:
                token_empty = False
            else:
                token_empty = True
        except:
            token_empty = True
        
        if token_empty:
            cmd.send_token_email(receiver, root, config, 'reset')
            
            ########## ACCESS TO RESET PW ###########
            self.hide()
            reset_sc.show(True)
            return
            #########################################

        else:
            ########## ACCESS TO RESET PW ###########
            self.hide()
            reset_sc.show(False)
            return
            #########################################

    def callback_create(self, b):
        # Redirect to create_screen
        self.user_info = {"ID" : self.username_entry.value, "PW" : self.userpw_entry.value}
        
        ########## ACCESS TO CREATE ACOUNT ###########
        self.hide()
        create_sc.show()
        ##############################################
        

    

        
        
class reset_pw_screen:
    def __init__(self):
        self.screen_title = w.HTML("<center"+sc_title+">Réinitialisation de mot de passe</center>")
        
        self.token_entry = w.Text(description="Code secret")
        self.new_password_entry = w.Password(description="Nouveau mot de passe")
        self.new_password_entry_2 = w.Password(description="Confirmer votre nouveau mot de passe")
        self.submit = w.Button(description="Valider")
        
        # Callback functions
        self.submit.on_click(self.callback_check_submit)
        
        #Images
        self.solo_pote = w.HTML("<center><img src='solo_pote.png' width=150 height=150 align='center'/></center>")
    
    def init_mess(self):
        #Mesages
        self.token_invalid = w.HTML("<center"+err_mess+">Mauvais message secret, veuillez réessayer</center>")
        self.pw_no_match = w.HTML("<center"+err_mess+">Les mots de passes ne correspondent pas</center>")
        self.mail_send = w.HTML("<center"+err_mess+">Nous vous avons envoyé un Email de réinitialisation à " + str(self.user['ID']) + "</center>")
        self.pw_too_short = w.HTML("<center"+err_mess+">Votre mot de passe doit contenir au moins " + str(config["min_pw_length"]) + " caractères</center>")
        
    def show(self, is_mail_sent):
        self.user = auth_sc.user_info
        self.__init__()
        self.init_mess()
        if is_mail_sent:
            w_list = [self.screen_title, self.mail_send, self.token_entry, self.new_password_entry, self.new_password_entry_2, self.submit]
        else:
            w_list = [self.screen_title, self.token_entry, self.new_password_entry, self.new_password_entry_2, self.submit]
        out = cmd.center_widget(w_list)
        with stdout:
            display(self.solo_pote)
            display(out)
        display(stdout)
    
    def hide(self):
        self.token_entry.close()
        self.new_password_entry.close()
        self.new_password_entry_2.close()
        self.submit.close()
        self.screen_title.close()
        self.solo_pote.close()
        self.hide_mess()
        
    def show_mess(self, mess):
        if mess == 'token_invalid':
            with stdout:
                self.hide_mess()
                display(self.token_invalid)
        if mess == "pw_no_match":
            with stdout:
                self.hide_mess()
                display(self.pw_no_match)
        if mess == "mail_sent":
            with stdout:
                self.hide_mess()
                display(self.mail_send)
        if mess=="pw_too_short":
            with stdout:
                self.hide_mess()
                display(self.pw_too_short)
            
    
    def hide_mess(self):
        self.token_invalid.close()
        self.pw_no_match.close()
        self.mail_send.close()
        self.pw_too_short.close()
        self.init_mess()
        
    def callback_check_submit(self, b):
        bdd = cmd.refresh(root)
        if self.token_entry.value == bdd.loc[self.user['ID'], 'TOKEN']:
            if self.new_password_entry.value == self.new_password_entry_2.value:
                if len(str(self.new_password_entry.value)) >= config["min_pw_length"]:
                    bdd.loc[self.user['ID'], 'TOKEN'] = str()
                    bdd.loc[self.user['ID'], 'USER_PW'] = self.new_password_entry.value
                    filepath = '\\'.join([root, 'data', 'BDD.json'])
                    bdd.to_json(filepath)

                    # Redirect to auth_screen

                    ###### ACCESS TO AUTH SCREEN #####
                    self.hide()
                    auth_sc.show()
                    ##################################
                else:
                    self.show_mess("pw_too_short")
                
            else:
                self.show_mess("pw_no_match")
                return
        else:
            self.show_mess('token_invalid')



class create_account_screen:
    def __init__(self):
        self.screen_title = w.HTML("<center"+sc_title+">Création de compte</center>")
        
        self.user_mail = w.Text(description="Adresse mail", value="@gmail.com")
        self.pw_entry = w.Password(description="Mot de passe")
        self.pw_entry_2 = w.Password(description="Mot de passe")
        self.submit_but = w.Button(description="Créer")
        self.token_entry = w.Text(description="Code secret")
        self.confirm_creation = w.Button(description="Terminer")
        self.back_to_authscreen = w.Button(description="Retour")
        
        #CAllback functions
        self.confirm_creation.on_click(self.callback_check_token)
        self.back_to_authscreen.on_click(self.back)
        self.submit_but.on_click(self.callback_check_and_send)
        
        #Images
        self.solo_pote = w.HTML("<center><img src='solo_pote.png' width=150 height=150 align='center'/></center>")
        
    def init_mess(self):
        # Messages
        self.mail_not_valid = w.HTML("<center"+err_mess+">Veuillez entrer une adresse Gmail valide (...@gmail.com)</center")
        self.pw_no_match = w.HTML("<center"+err_mess+">Les mots de passes ne correspondent pas</center>")
        self.mail_send = w.HTML("<center"+err_mess+">Nous vous avons envoyé un Email de confirmation à " + str(self.user_mail.value)+ "</center>")
        self.user_already_in_bdd = w.HTML("<center>Vous avez déja un compte dans Les Polypotes</center>")
        self.token_no_match = w.HTML("<center"+err_mess+">Votre code secret est incorrect. Vous pouvez recliquer sur 'Créer' pour renvoyer un code secret</center>")
        self.pw_too_short = w.HTML("<center"+err_mess+">Votre mot de passe doit contenir au moins " + str(config["min_pw_length"]) + " caractères</center>")
        
    def show(self):
        self.__init__()
        self.init_mess()
        
        box_list = [self.screen_title, 
                    self.user_mail, 
                    self.pw_entry, 
                    self.pw_entry_2, 
                    [self.back_to_authscreen, self.submit_but]]
        show_box = cmd.center_widget(box_list)
        
        with stdout:
            display(self.solo_pote)
            display(show_box)

    def hide(self):
        self.user_mail.close()
        self.pw_entry.close()
        self.pw_entry_2.close()
        self.submit_but.close()
        self.mail_send.close()
        self.token_entry.close()
        self.confirm_creation.close()
        self.back_to_authscreen.close()
        self.screen_title.close()
        self.solo_pote.close()
        self.hide_mess()

    def show_mess(self, mess):
        if mess == 'mail_not_valid':
            with stdout:
                self.hide_mess()
                display(self.mail_not_valid)
        if mess == "pw_no_match":
            with stdout:
                self.hide_mess()
                display(self.pw_no_match)
        if mess == "mail_sent":
            with stdout:
                self.hide_mess()
                display(self.mail_send)
        if mess == "user_already_in_bdd":
            with stdout:
                self.hide_mess()
                display(self.user_already_in_bdd)
        if mess == 'token_no_match':
            with stdout:
                self.hide_mess()
                display(self.token_no_match)
        if mess == "pw_too_short":
            with stdout:
                self.hide_mess()
                display(self.pw_too_short)

    def hide_mess(self):
        self.mail_not_valid.close()
        self.pw_no_match.close()
        self.mail_send.close()
        self.pw_too_short.close()
        self.init_mess()
    
    def show_child(self):
        self.tmp = self.user_mail.value, self.pw_entry.value, self.pw_entry_2.value
        self.hide()
        self.__init__()
        self.user_mail.value, self.pw_entry.value, self.pw_entry_2.value = self.tmp[0], self.tmp[1], self.tmp[2]
        self.init_mess()
        
        
        sub_list = [self.screen_title, 
                    self.mail_send, 
                    self.token_entry, 
                    [self.back_to_authscreen, self.confirm_creation]]
        sub_out = cmd.center_widget(sub_list)
        with stdout:
            display(self.solo_pote)
            display(sub_out)

    def callback_check_and_send(self, b):
        bdd = cmd.refresh(root)
        if cmd.check_mail_domain(self.user_mail.value, config):
            if not self.user_mail.value in bdd.index:
                if self.pw_entry.value == self.pw_entry_2.value:
                    if len(str(self.pw_entry.value)+'a') > config["min_pw_length"]:
                        self.show_child()
                        token = cmd.send_token_email(self.user_mail.value, root, config, "create")
                        user_info = {'ID': self.user_mail.value,
                                    'TOKEN': token,
                                    'USER_PW': self.pw_entry.value}
                        bdd = cmd.add_user(bdd, user_info)
                        cmd.refresh(root, bdd)
                        return
                    
                    else:
                        self.show_mess("pw_too_short")
                        return
                else:
                    self.show_mess('pw_no_match')
                    return
            else:
                self.show_mess('user_already_in_bdd')
                return
        else:
            self.show_mess('mail_not_valid')
            return

    def callback_check_token(self, b):
        bdd = cmd.refresh(root)
        if bdd.loc[self.user_mail.value, "TOKEN"] == self.token_entry.value:
            bdd.loc[self.user_mail.value, "TOKEN"] = ""
            cmd.refresh(root, bdd)
            # Redirect to auth_screen
            print('good')
            
            #### ACCES TO AUTH SCREEN #####
            self.hide()
            auth_sc.show()
            ###############################
            
        else:
            self.show_mess('token_no_match')
    
    def back(self, b):
        
        ##### ACCESS TO AUTH SCREEN #####
        self.hide()
        auth_sc.show()
        #################################



        
        

In [8]:
class signin_screen:
    def __init__(self):
        # Main widgets
        self.screen_title = w.HTML("<center"+sc_title+">Les questions</center>")
        
        self.lab_quest1 = w.Label(value=questions['q1'])
        self.entry_quest1 = w.Text(description="Réponse :")
        self.lab_quest2 = w.Label(value=questions['q2'])
        self.entry_quest2 = w.Text(description="Réponse :")
        self.lab_quest3 = w.Label(value=questions['q3'])
        self.entry_quest3 = w.Text(description="Réponse :")
        self.valide_answer = w.Button(description="Valider")
        
        self.spacing = w.HTML("<center style='padding-top: 20px'> </center>")
        
        #callback functions
        self.valide_answer.on_click(self.callback_check_answers)
    
    def init_mess(self):
        # messages
        self.wrong_ans_1 = w.HTML("<center"+err_mess+">Mauvaise réponse 1</center>")
        self.wrong_ans_2 = w.HTML("<center"+err_mess+">Mauvaise réponse 2</center>")
        self.wrong_ans_3 = w.HTML("<center"+err_mess+">Mauvaise réponse 3</center>")
        
        
        
    def show(self):
        self.__init__()
        self.init_mess()
        
        box_list = [self.screen_title, 
                    self.spacing, 
                    self.lab_quest1, 
                    self.entry_quest1, 
                    self.spacing, 
                    self.lab_quest2,
                    self.entry_quest2, 
                    self.spacing, 
                    self.lab_quest3,
                    self.entry_quest3,
                    self.spacing,
                    self.spacing,
                    self.valide_answer]
        disp_box = cmd.center_widget(box_list)
        
        with stdout:
            display(disp_box)
        display(stdout)
    
    def show_mess(self, mess):
        if mess == 'wrong1':
            with stdout:
                self.hide_mess()
                display(self.wrong_ans_1)
        if mess == 'wrong2':
            with stdout:
                self.hide_mess()
                display(self.wrong_ans_2)
        if mess == 'wrong3':
            with stdout:
                self.hide_mess()
                display(self.wrong_ans_3)
        
        
    def hide(self):
        self.lab_quest1.close()
        self.entry_quest1.close()
        self.lab_quest2.close()
        self.entry_quest2.close()
        self.lab_quest3.close()
        self.entry_quest3.close()
        self.valide_answer.close()
        self.screen_title.close()
        self.spacing.close()
        self.hide_mess()
    
    def hide_mess(self):
        self.wrong_ans_1.close()
        self.wrong_ans_2.close()
        self.wrong_ans_3.close()
        self.init_mess()
    
    def callback_check_answers(self, b):
        answers = {'a1' : self.entry_quest1.value,
                   'a2' : self.entry_quest2.value,
                   'a3' : self.entry_quest3.value}
        
        res = cmd.check_answers(answers) # result is dict with same entries as 'answers'
        
        all_good = True
        if res['a1'] == False:
            all_good = False
            self.show_mess('wrong1')
        if res['a2'] == False:
            all_good = False
            self.show_mess('wrong2')
        if res['a3'] == False:
            all_good = False
            self.show_mess('wrong3')
        
        if all_good == False:
            return
        else:
            bdd = cmd.refresh(root)
            user_id = auth_sc.user_info
            bdd.loc[user_id['ID'], 'IS_POLYTECH'] = 'oui'
            cmd.refresh(root, bdd)
            
            ##### ACCESS TO TUTO #####
            self.hide()
            tutoriel_sc.show()
            ##########################
    


class tuto_screen:
    def __init__(self):
        # Main widgets
        self.screen_title = w.HTML("<center"+sc_title+">Tutoriel</center>")
        self.actual_screen = 1
        self.screen_nb = len(tuto.keys())
        
        # Dynamic display
    def init_dynamic(self):
        self.screen_title = w.HTML("<center"+sc_title+">Tutoriel</center>")
        self.message = w.HTML("a")
        self.next_but = w.Button(description="Suivant")
        self.next_but.on_click(self.callback_next)
        self.back_but = w.Button(description='Précédent')
        self.back_but.on_click(self.callback_back)
    
    
    def show(self):
        self.__init__()
        self.init_dynamic()
        self.actual_screen = 1
        self.message.value = "<center>"+tuto['screen1']+"</center>"
        
        w_list = [self.screen_title,
                    self.message,
                    self.next_but]
        w_box = cmd.center_widget(w_list)
        
        with stdout:
            display(w_box)
        display(stdout)
    
    def hide_curr_mess(self):
        self.message.close()
        self.back_but.close()
        self.next_but.close()
        self.screen_title.close()
        self.init_dynamic()

    def callback_next(self, b):
        self.hide_curr_mess()
        self.actual_screen += 1 
        
        if self.actual_screen > self.screen_nb:
            
            ##### ACCESS TO inscr_screen #####
            # auto_hide
            inscr_sc.show()
            ##################################
            
          
        if self.actual_screen <= self.screen_nb:
            self.message = w.HTML("<center>"+ tuto['screen'+str(self.actual_screen)] +"</center>")
            
            w_list = [self.screen_title,
                      self.message, 
                      [self.back_but, self.next_but]]
            
            w_box = cmd.center_widget(w_list)
            
            with stdout:
                display(w_box)
            display(stdout)
    
    def callback_back(self, b):
        self.hide_curr_mess()
        self.actual_screen -= 1
        if self.actual_screen > 1:
            self.message.value = "<center>"+ tuto['screen'+str(self.actual_screen)] +"</center>"
            w_list = [self.screen_title, 
                      self.message, 
                      [self.back_but, self.next_but]]
            w_box = cmd.center_widget(w_list)
            with stdout:
                display(w_box)
            display(stdout)
        else:
            self.actual_screen = 1
            self.message.value =  "<center>"+ tuto['screen'+str(self.actual_screen)] +"</center>"
            w_list = [
                self.screen_title,
                self.message, 
                self.next_but
            ]
            w_box = cmd.center_widget(w_list)
            with stdout:
                display(w_box)
            display(stdout)

            
            
            
            
class inscr_screen():
    def __init__(self):
        # Misc config
        promo_list = [i for i in range(inscr_form_opt["promo_list_bounds"][0], inscr_form_opt["promo_list_bounds"][1] +1)]
        self.df_ville_fr = cmd.read_communes(root)
        self.pays = cmd.read_liste_pays_monde(root)
        now = datetime.datetime.now()
        self.date = datetime.datetime.timestamp(now)
        
        
        # Main widgets
        self.spacing = w.HTML("<center style='padding-top: 20px'> </center>")
        self.screen_title = w.HTML("<center"+sc_title+">Inscription</center>")
        self.lab_name = w.HTML(value="<center style='padding-right: 20px'>Nom Prénom *</center>")           # mandaotry
        self.entry_name = w.Text(layout=w.Layout(width='200px'))
        self.lab_fbname = w.HTML(value="<center style='padding-right: 5px'>Pseudo Facebook</center>")
        self.entry_fbname = w.Text(layout=w.Layout(width='200px'))
        self.lab_promo = w.HTML(value="<center style='padding-right: 58px'>Promo *</center>")               # Mandatory
        self.entry_promo = w.Dropdown(options=promo_list, 
                                      value=promo_list[0],
                                     layout=w.Layout(width='200px'))   # mandatory
        self.lab_ecole = w.HTML(value="<center style='padding-right: 66px'>Ecole *</center>")
        self.entry_ecole = w.Dropdown(options=inscr_form_opt["polytech_list"],
                                     layout=w.Layout(width='200px')) #mandatory
        self.lab_ville = w.HTML(value="<center style='padding-right: 72px'>Ville *</center>") 
        self.entry_ville = w.Combobox(placeholder='Entrez une ville',
                                      options=self.df_ville_fr['nom_commune'].values.tolist(),
                                      ensure_option=False,
                                     layout=w.Layout(width='200px')) # mandatory
        self.lab_code_post = w.HTML(value="<center style='padding-right: 29px'>Code postal *</center>")
        self.entry_code_post = w.Text(layout=w.Layout(width='200px')) # mandatory
        self.lab_pays = w.HTML(value="<center style='padding-right: 72px'>Pays *</center>")
        self.entry_pays = w.Combobox(placeholder='Entrez un pays',
                                      options=self.pays['PAYS'].values.tolist(),
                                      ensure_option=False,
                                    layout=w.Layout(width='200px')) # mandatory
        self.deso_que_fr = w.Label(value="Nous n'avons pas encore intégré toutes les villes de tous les pays")
        
        #Buttons
        self.submit_but = w.Button(description="Soumettre")
        self.submit_but.on_click(self.callback_update) 
        #self.add_city_but = w.Button(description="Proposer une ville")
    
    def init_mess(self):
        self.mess_name = w.Label(value="<center"+err_mess+">Champ 'Nom Prénom' invalide</center>")
        self.mess_ville = w.Label(value="<center"+err_mess+">Champ 'Ville' invalide</center>")
        self.mess_cp = w.Label(value="<center"+err_mess+">Champ 'Code Postal' invalide</center>")
        self.mess_pays = w.Label(value="<center"+err_mess+">Champ 'Pays' invalide</center>")
        
        
    def show(self):
        self.__init__()
        self.init_mess()
        
        w_list = [
            self.screen_title,
            [self.lab_name, self.entry_name], 
            self.spacing,
            [self.lab_fbname,self.entry_fbname],
            self.spacing,
            [self.lab_promo,self.entry_promo],
            self.spacing,
            [self.lab_ecole,self.entry_ecole],
            self.spacing,
            [self.lab_ville,self.entry_ville],
            self.spacing,
            [self.lab_code_post, self.entry_code_post],
            self.spacing,
            [self.lab_pays, self.entry_pays],
            self.deso_que_fr,
            self.spacing,
            self.spacing,
            self.submit_but]
        
        w_box = cmd.center_widget(w_list)
        
        with stdout:
            display(w_box)
        display(stdout)   
        
    def hide(self):
        self.lab_name.close() 
        self.entry_name.close()
        self.lab_fbname.close()
        self.entry_fbname.close() 
        self.lab_promo.close()
        self.entry_promo.close()
        self.lab_ecole.close()
        self.entry_ecole.close()
        self.lab_ville.close()
        self.entry_ville.close()
        self.submit_but.close()
        self.screen_title.close()
        self.lab_code_post.close()
        self.entry_code_post.close()
        self.lab_pays.close()
        self.entry_pays.close()
        self.deso_que_fr.close()
        self.spacing.close()
        #self.hide_mess()
    
    def show_mess(self, mess):
        if mess == 'entry_name':
            with stdout:
                self.hide_mess()
                display(self.mess_name)
            display(stdout)
        if mess == 'ville':
            with stdout:
                self.hide_mess()
                display(self.mess_ville)
            display(stdout)
        if mess == "entry_code_post":
            with stdout:
                self.hide_mess()
                display(self.mess_cp)
            display(stdout)
        if mess == "entry_pays":
            with stdout:
                self.hide_mess()
                display(self.mess_pays)
            display(stdout)
    
    def hide_mess(self):
        self.mess_name.close()
        self.mess_ville.close()
        self.mess_cp.close()
        self.mess_pays.close()
        self.init_mess()
   
    def callback_update(self, b):
        
        # CHECK ALL MANDATORY FIELDS ARE FILLED
        flag = True
        if cmd.is_empty(self.entry_name.value):
            self.show_mess('entry_name')
            flag = False
            
        if cmd.is_empty(self.entry_ville.value):
            self.show_mess('ville')
            flag = False
        
        if cmd.is_empty(self.entry_code_post.value):
            self.show_mess('entry_code_post')
            flag = False
        
        if cmd.is_empty(self.entry_pays.value):
            self.show_mess('entry_pays')
            flag = False
        
        if flag == False:
            return
            
        ##### TODO: A revoir ##############################
        # find CITY latlong
        if self.entry_pays.value.lower().replace(" ","") == 'france':
            lat, long, ret = cmd.find_latlong(self.df_ville_fr, self.entry_code_post.value)
        elif self.entry_pays.value.lower().replace(" ","") == 'belgique':
            print()
        else:
            self.show_mess('ville_not_found')
            # Que faire
        ####################################################
        
        # Write in BDD
        bdd = cmd.refresh(root)
        update_dic = {"USER_ID"  : str(self.entry_name.value),
                      'PSEUDO_FB': str(self.entry_fbname.value),
                      "PROMO"    : str(self.entry_promo.value),
                      "ECOLE"    : str(self.entry_ecole.value),
                      "VILLE"    : str(self.entry_ville.value),
                      "INSCRIPTION": int(self.date),
                      "LAT"      : lat,
                      'LONG'     : long,
                      'IS_BANNED': 'non'}
            
        bdd, error = cmd.update_user(bdd, update_dic, glob_usr_mail)
        cmd.refresh(root, bdd)
        
        if error == 0:
            update_dic = {"USER_MAIL" : glob_usr_mail,
                          "USER_ID" : str(self.entry_name.value),
                          "CONTENT_ID" : "inscription",
                          "CONTENT" : ""}
            error = cmd.add_log(root, update_dic, glob_usr_mail)
            if error == 0:
                
                #### ACCESS TO MAIN SCREEN ####
                self.hide()
                main_sc.show()
                ###############################
                
        else:
            return
        
        
        
        
       


 

In [15]:
class main_screen:
    def __init__(self):
        bdd = cmd.refresh(root)
        
        #Main widgets
        self.map = folium.Map(location=[45, 1.9], zoom_start=2)
        self.map_disp = folium.Figure(width=1130, height=500)
        self.coord_table = w.HTML(" ")
        self.logs_table = w.HTML(" ")
        self.search_bar = w.Combobox(placeholder='Recherche',
                                      options=bdd['USER_ID'].values.tolist(),
                                      ensure_option=True,
                                      layout={'width':'100%'}) # mandatory
        self.search_but = w.Button(description=emoji.emojize(':magnifying_glass_tilted_left:') + "  Rechercher",
                                  layout={'padding':"0px 0px 0px 0px",
                                         'float':'right'})# top, right, bottom, left
        
        self.disconnect_but = w.Button(description=emoji.emojize(':red_circle:') + " Se Déconnecter",
                                       layout={'width':'auto'})
        self.setting_but = w.Button(description=emoji.emojize(':gear:') + " Paramètres Compte",
                                    layout={'width':'auto'})
        
        # Labels and other
        self.lab_coord_table = w.HTML("<h2>Coordonnées</h2>")
        self.screen_title = w.HTML("<center"+sc_title+">Les Polypotes</center>", layout={'grid_area':'sc_title'})
        
        #Images
        self.solo_pote = w.HTML("<center><img src='solo_pote.png' width=100 height=100 align='center'/></center>")
        
        # Callbacks
        self.search_but.on_click(self.callback_find)
        self.disconnect_but.on_click(self.callback_disconnect)
        self.setting_but.on_click(self.callback_setting)
        
        
    def init_dynamic(self):
        print()
    
    def show(self):
        self.__init__()
        self.init_dynamic()
        
        self.fill_map()
        self.fill_coord_table()
        #self.fill_logs_table()
        
        
        # Define outputs
        self.coord_table_out = w.Output(layout={'width': 'auto', 'grid_area':'table'})
        self.map_out = w.Output(layout={'width': 'auto', 'grid_area':'map'})
        self.coord_table_out.append_display_data(self.coord_table)
        self.map_out.append_display_data(self.map_disp)
        
        # Organize widgets
        self.search_disp = w.HBox(children=[self.search_bar, 
                                            self.search_but], 
                                  layout={'width':'1135px',
                                          'grid_area':'search'})
        self.left_sider = w.VBox(children=[self.solo_pote,
                                      self.screen_title,
                                      #self.setting_but,
                                      self.disconnect_but],
                            layout={'width':'auto',
                                    'grid_area' : 'left',
                                    'padding' : "0px 30px 0px 0px"}) # top, right, bottom, left
        
        self.grid_out = w.GridBox(children=[self.coord_table_out,
                                            self.map_out,
                                            self.left_sider,
                                            self.search_disp],
                                  layout=w.Layout(grid_template_rows='auto auto auto auto',
                                                  grid_template_columns='15% 35% 30% 20%',
                                                  grid_template_areas='''
                                                  "left search search ."
                                                  "left map   map ."
                                                  "left map   map ."
                                                  ". table table ."
                                                  ''')
                                 )
        
        
        with stdout:
            display(self.grid_out)
        display(stdout)

    def hide(self):
        self.grid_out.close()
        self.left_sider.close()
        self.map_out.clear_output()
        self.coord_table_out.clear_output()
        self.solo_pote.close()
        self.screen_title.close()
        self.lab_coord_table.close()
        self.disconnect_but.close()
        self.search_but.close()
        self.search_bar.close()
        self.logs_table.close()
    
    def callback_disconnect(self, b):
        glob_usr_mail = ""
        self.hide()
        auth_sc.show()
    
    def callback_find(self, b):
        bdd = cmd.refresh(root)
        keyword = str(self.search_bar.value)
        ind = bdd[bdd['USER_ID']==keyword].index.values
        
        
    
    def callback_setting(self, b):
        self.hide()
        param_sc.show()
        
        
    
    def fill_map(self, filtred=False):
        bdd = cmd.refresh(root)
        self.map = cmd.create_map(bdd, self.map)
        self.map.add_to(self.map_disp)
                
    def fill_coord_table(self, filtred=False):
        bdd = cmd.refresh(root)
        self.coord_table = cmd.create_coord_table(bdd)

    def fill_logs_table(self):
        logs = cmd.read_logs(root)
        logs_disp = logs
        self.logs_table = w.HTML(tabulate.tabulate(logs_disp, headers=logs_disp.columns, tablefmt="html"))
        
        

class param_screen():
    def __init__(self):
        
        # Main widgets
        self.screen_title = w.HTML("<center"+sc_title+">Paramètres du compte</center>")
        
        self.lab_visible = w.HTML("Visibilité", 
                                 layout={'padding': "0px 50px 0px 0px"})# top, right, bottom, left
        self.visible_but = w.ToggleButtons(options=[' Visible ', ' Caché '],
                                           tooltips=['Etre caché permet aux autres utilisateurs de ne pas voir vos informations'],
                                           icons=['check', 'close'])
        self.lab_del_account = w.HTML("Supprimer le compte",
                                     layout={'padding': "0px 50px 0px 0px"})
        self.del_account_but = w.Button(description="Supprimer",
                                        button_style='danger')
        
        self.return_but = w.Button(description="Enregistrer et quitter",
                                   button_style="success")
        
        
        
        #Callbacks
        self.visible_but.observe(self.callback_visible, 'Caché')
        self.del_account_but.on_click(self.callback_del_account)
        self.return_but.on_click(self.callback_return)
        
        #Images
        self.solo_pote = w.HTML("<center><img src='solo_pote.png' width=150 height=150 align='center'/></center>")
        
    def show(self):
        w_list = [self.solo_pote,
                  self.screen_title,
                 [self.lab_visible, self.visible_but],
                 [self.lab_del_account, self.del_account_but],
                 self.return_but]
        w_box = cmd.center_widget(w_list)
        
        with stdout:
            display(w_box)
        display(stdout)
    
    def hide(self):
        self.screen_title.close()
        self.lab_visible.close()
        self.visible_but.close()
        self.lab_del_account.close()
        self.del_account_but.close()
        self.return_but.close()
        self.solo_pote.close()
        
        self.__init__()
        
    def callback_visible(self, b):
        
        print()
    
    def callback_del_account(self, b):
        
        print()
    
    def callback_return(self, b):
        self.hide()
        main_sc.show()
        

class del_account():
    def __init__(self):
        self.screen_title = w.HTML("<center"+sc_title+">Supprimer le compte ?</center>")
        self.solo_pote = w.HTML("<center><img src='solo_pote.png' width=150 height=150 align='center'/></center>")
        self.del_but = w.Button(description="Supprimer")
        self.cancel = w.Button(description="Annuler")
        
        self.del_but.onclick(self.callback_delete)
        self.cancel.on_click(self.callback_cancel)
        
    def show(self):
        w_list = [self.solo_pote,
                 self.Screen_title,
                 [self.cancel, self.del_but]]
        w_box = cmd.center_widget(w_list)
        
        with stdout:
            display(w_box)
        display(stdout)
    
    def hide(self):
        self.screen_title.close()
        self.solo_pote.close()
        self.del_but.close()
        self.cancel.close()
    
    def callback_cancel(self, b):
        self.hide()
        param_sc.show()
    
    def callback_delete(self, b):
        bdd = cmd.refresh(root)
        bdd = bdd.drop(index=[glob_usr_mail])
        cmd.refresh(root, bdd)
        
        glob_usr_mail
        self.hide()
        auth_sc.show()
        

In [16]:
auth_sc = auth_screen()
reset_sc = reset_pw_screen()
create_sc = create_account_screen()

sign_sc = signin_screen() #questions
tutoriel_sc = tuto_screen()
inscr_sc = inscr_screen()

main_sc = main_screen()
param_sc = param_screen()

#inscr_sc.show()
main_sc.show()

#os.system("voila --debug")

#auth_sc.show()


Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "GridBox(children=(Output(layout=Layout…

## 